# Preprocessing

In [22]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from datetime import date, time, datetime, timedelta
import matplotlib.dates as matdates
import warnings

# Change filename as appropriate
filename = "Hypochilus Monitor14 Edited"
df = pd.read_csv(filename + ".txt", index_col = 0, header = None, delimiter="\t")
#df['Date-Time'] = pd.to_datetime(df.index, utc=True)

#df.set_index('Date-Time', inplace=True)

#display(df)

# This function creates the column labels into a list 

def create_column_labels(date_time_label, light_label, spider_count):
    # Create column labels
    column_names = []
    column_names.append(date_time_label)
    column_names.append(light_label)
    
    # Create spider labels 
    for x in range(1, spider_count + 1):
        column_names.append('Spider ' + str(x))
        
    return column_names

def insert_row(idx, df, df_insert):
    dfA = df.iloc[:idx, ]
    dfB = df.iloc[idx:, ]
    df = pd.concat([dfA, df_insert, dfB], ignore_index = True)
    return df

def generate_time_series(previous_datetime, count):
    # result = [('2022-10-21 13:00', NaN, NaN, ....),
    #           ('2022-10-21 13:01', NaN, NaN, ....),
    #           ...]
    result = []
    for i in range(count):
        previous_datetime = previous_datetime + timedelta(seconds = 60)
        result.append(
            (previous_datetime.strftime("%Y-%m-%d %H:%M:%S"), np.nan,
                                np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
                                np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
                                np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
                                np.nan, np.nan)
        )
    return result
    
def autoFillDataFrame(df):
    index = 0
    #current_datetime = datetime.strptime(df["Date-Time"][0], "%Y-%m-%d %H:%M:%S")
    current_datetime = df["Date-Time"][0]
    previous_datetime = datetime.combine(current_datetime, time.min) - timedelta(seconds = 60)
    while index < len(df.index):
        #current_datetime = datetime.strptime(df["Date-Time"][index], "%Y-%m-%d %H:%M:%S")
        current_datetime = df["Date-Time"][index]
        time_diff = int((current_datetime - previous_datetime).total_seconds())
        if time_diff > 60:
            count = int((time_diff - 60) / 60)
            df = insert_row(index, df, pd.DataFrame(
                generate_time_series(previous_datetime, count),
                columns = df.columns.values))
            index = index + count
        previous_datetime = current_datetime
        index = index + 1
    if not (current_datetime.hour == 23 and current_datetime.minute == 59 ):
        count = (23 - current_datetime.hour) * 60 + (59 - current_datetime.minute)
        df = insert_row(index, df, pd.DataFrame(
                generate_time_series(previous_datetime, count),
                columns = df.columns.values))
    return df

# This function creates a new .csv file with columns labeled Light and Spider name and Date-Time as
# the index

def txt_cleaning_to_csv(df):
    # Change spider count, your light column index number, and date-time label as you see fit
    number_of_spiders = 32
    light_column = 9
    date_time_label = 'Date-Time'
    column_names = create_column_labels(date_time_label, 'Light', number_of_spiders)

    # Keep the last number of spiders columns, the light column, and the first 2 columns, the rest will be removed
    last_df = df.iloc[:, -number_of_spiders:]
    first_df = df.iloc[:, :2]
    light_df = df.iloc[:, light_column - 1]
    date_time_df = first_df[1].astype(str) + ' ' + first_df[2].astype(str)

    # Concatenate the first and last sections of the dataframe
    tempdf = pd.concat([date_time_df, light_df, last_df], axis = 1)

    # Save dataframe as .csv file and set Date and Time columns into datetime format
    
    tempdf.to_csv("Hypochilus Monitor 14 Temp.csv", header = column_names, index = False)
    tempdf = pd.read_csv("Hypochilus Monitor 14 Temp.csv", parse_dates = [date_time_label])
    newdf = autoFillDataFrame(tempdf)
    display(newdf)
    newdf.to_csv("Hypochilus Monitor 14.csv", float_format="%d")
    
    #newdf.set_index(date_time_label, inplace = True)
    #newdf.index = pd.to_datetime(newdf.index)
    
    return newdf
    
df = txt_cleaning_to_csv(df)

,Date-Time,Light,Spider 1,Spider 2,Spider 3,Spider 4,Spider 5,Spider 6,Spider 7,Spider 8,...,Spider 23,Spider 24,Spider 25,Spider 26,Spider 27,Spider 28,Spider 29,Spider 30,Spider 31,Spider 32
0,2022-10-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-10-12 00:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-12 00:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-10-12 00:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-10-12 00:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40315,2022-11-08 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40316,2022-11-08 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40317,2022-11-08 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40318,2022-11-08 23:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Splitting

In [5]:
def splitDF(df):
    df_split = df.copy()
    lights_turn_off = df[df['Light'].diff() == -1].index[-1]
    lights_turn_on = df[df['Light'].diff() == 1].index[-1]
    LD = df_split[df_split.index < lights_turn_off]
    LD.to_csv(filename + ' LD.csv')
    DD = df_split[df_split.index >= lights_turn_off]
    DD.to_csv(filename + ' DD.csv')
    display(LD, DD)
    
    return LD, DD

splitDF(df)

,Light,Spider 1,Spider 2,Spider 3,Spider 4,Spider 5,Spider 6,Spider 7,Spider 8,Spider 9,...,Spider 23,Spider 24,Spider 25,Spider 26,Spider 27,Spider 28,Spider 29,Spider 30,Spider 31,Spider 32
Date-Time,,,,,,,,,,,,,,,,,,,,,
2022-10-12 15:43:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-12 15:44:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-12 15:45:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-12 15:46:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-12 15:47:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-18 19:57:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-18 19:58:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-18 19:59:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Light,Spider 1,Spider 2,Spider 3,Spider 4,Spider 5,Spider 6,Spider 7,Spider 8,Spider 9,...,Spider 23,Spider 24,Spider 25,Spider 26,Spider 27,Spider 28,Spider 29,Spider 30,Spider 31,Spider 32
Date-Time,,,,,,,,,,,,,,,,,,,,,
2022-10-18 20:02:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-18 20:03:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-18 20:04:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-18 20:05:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-18 20:06:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-08 13:46:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-11-08 13:47:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-11-08 13:48:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(                     Light  Spider 1  Spider 2  Spider 3  Spider 4  Spider 5  \
 Date-Time                                                                      
 2022-10-12 15:43:00      1       0.0       0.0       0.0       0.0       0.0   
 2022-10-12 15:44:00      1       0.0       0.0       0.0       0.0       0.0   
 2022-10-12 15:45:00      1       0.0       0.0       0.0       0.0       0.0   
 2022-10-12 15:46:00      1       0.0       0.0       0.0       0.0       0.0   
 2022-10-12 15:47:00      1       0.0       0.0       0.0       0.0       0.0   
 ...                    ...       ...       ...       ...       ...       ...   
 2022-10-18 19:57:00      1       0.0       0.0       0.0       0.0       0.0   
 2022-10-18 19:58:00      1       0.0       0.0       0.0       0.0       0.0   
 2022-10-18 19:59:00      1       0.0       0.0       0.0       0.0       0.0   
 2022-10-18 20:00:00      1       0.0       0.0       0.0       0.0       0.0   
 2022-10-18 20:01:00      1 

In [23]:
def splitDF(df):
    df_split = df.copy()
    lights_turn_off = df[df['Light'].diff() == -1].index[-1]
    lights_turn_on = df[df['Light'].diff() == 1].index[-1]
    LD = df_split[df_split.index < lights_turn_off]
    LD.to_csv(filename + ' LD.csv')
    DD = df_split[df_split.index >= lights_turn_off]
    DD.to_csv(filename + ' DD.csv')
    display(LD, DD)
    
    return LD, DD

splitDF(df)

,Date-Time,Light,Spider 1,Spider 2,Spider 3,Spider 4,Spider 5,Spider 6,Spider 7,Spider 8,...,Spider 23,Spider 24,Spider 25,Spider 26,Spider 27,Spider 28,Spider 29,Spider 30,Spider 31,Spider 32
0,2022-10-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-10-12 00:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-12 00:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-10-12 00:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-10-12 00:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9837,2022-10-18 19:57:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9838,2022-10-18 19:58:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9839,2022-10-18 19:59:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9840,2022-10-18 20:00:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Date-Time,Light,Spider 1,Spider 2,Spider 3,Spider 4,Spider 5,Spider 6,Spider 7,Spider 8,...,Spider 23,Spider 24,Spider 25,Spider 26,Spider 27,Spider 28,Spider 29,Spider 30,Spider 31,Spider 32
9842,2022-10-18 20:02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9843,2022-10-18 20:03:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9844,2022-10-18 20:04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9845,2022-10-18 20:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9846,2022-10-18 20:06:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40315,2022-11-08 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40316,2022-11-08 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40317,2022-11-08 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40318,2022-11-08 23:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(                Date-Time  Light  Spider 1  Spider 2  Spider 3  Spider 4  \
 0     2022-10-12 00:00:00    NaN       NaN       NaN       NaN       NaN   
 1     2022-10-12 00:01:00    NaN       NaN       NaN       NaN       NaN   
 2     2022-10-12 00:02:00    NaN       NaN       NaN       NaN       NaN   
 3     2022-10-12 00:03:00    NaN       NaN       NaN       NaN       NaN   
 4     2022-10-12 00:04:00    NaN       NaN       NaN       NaN       NaN   
 ...                   ...    ...       ...       ...       ...       ...   
 9837  2022-10-18 19:57:00    1.0       0.0       0.0       0.0       0.0   
 9838  2022-10-18 19:58:00    1.0       0.0       0.0       0.0       0.0   
 9839  2022-10-18 19:59:00    1.0       0.0       0.0       0.0       0.0   
 9840  2022-10-18 20:00:00    1.0       0.0       0.0       0.0       0.0   
 9841  2022-10-18 20:01:00    1.0       1.0       1.0       0.0       0.0   
 
       Spider 5  Spider 6  Spider 7  Spider 8  ...  Spider 23  Spider 24  